# Analysis of Most Winning Teams 2015/2016 Season

First, we analyze the most winning teams from the 2015/2016 season.  
Then, we run the same analysis for teams in England & Germany,  
and then on both England and Germany.  

## imports

In [18]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
import pyspark.sql.functions as fn
import pyspark.ml.feature as ft

from pyspark.sql.functions import isnan, when, count, col

Start spark session

In [19]:
spark = SparkSession.builder.master('local[4]').config("spark.executor.memory", "1g").config("spark.driver.memory", "2g").appName('spark_ml_soccer2').getOrCreate()
sc = spark.sparkContext

Read in parquet partition for 2015/2016 season

In [20]:
sparkDF2016 = spark.read.parquet("matchBySeason.parquet/season=2015%2F2016")

count the number of matches in 2015/2016 season

In [21]:
print("Total matches played in 2015/2016 season %s'" % sparkDF2016.count())
print("Distinct matches %s'" % sparkDF2016.distinct().count())

Total matches played in 2015/2016 season 3326'
Distinct matches 3326'


In [22]:
sparkDF2016.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- stage: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_team_goal: integer (nullable = true)
 |-- away_team_goal: integer (nullable = true)
 |-- goal: string (nullable = true)
 |-- shoton: string (nullable = true)
 |-- shotoff: string (nullable = true)
 |-- foulcommit: string (nullable = true)
 |-- card: string (nullable = true)
 |-- cross: string (nullable = true)
 |-- corner: string (nullable = true)
 |-- possession: string (nullable = true)
 |-- betting_odds_home: double (nullable = true)
 |-- betting_odds_draw: double (nullable = true)
 |-- betting_odds_away: double (nullable = true)



In [23]:
#look at the data via pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
sparkDF2016.limit(2).toPandas().head(2)

,_c0,id,country_name,league_name,stage,date,home_team,away_team,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,betting_odds_home,betting_odds_draw,betting_odds_away
0,23074,15680,Netherlands,Netherlands Eredivisie,5,2015-09-12 00:00:00,SC Cambuur,PSV,0,6,<goal><value><comment>p</comment><stats><penal...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>r</comment><stats><rcard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>40</comment><stats...,6.50,4.5,1.44
1,23075,15681,Netherlands,Netherlands Eredivisie,5,2015-09-12 00:00:00,PEC Zwolle,Excelsior,3,0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><stats...,1.73,3.8,4.50


## Analysis of all Matches from 2015/2016

In [24]:
from pyspark.sql import functions as fn
from pyspark.sql import Window

# obtain team level information
team_info = ['country_name', 'league_name', 'home_team','away_team','date', 'home_team_goal', 'away_team_goal', 'betting_odds_home', 'betting_odds_draw', 'betting_odds_away']
futbol_team_info = sparkDF2016.select(team_info)
futbol_team_info.show()

futbol_team_info = futbol_team_info.withColumn('win_team', fn.when(fn.col('home_team_goal') > fn.col('away_team_goal'), fn.col('home_team')).otherwise(fn.col('away_team')))
futbol_team_info = futbol_team_info.withColumn('game_year', fn.substring('date', 0, 4))
futbol_team_info.show()

# teams win count's
win_count = futbol_team_info.groupBy(fn.col('win_team').alias('team'), 'game_year').agg(fn.count('win_team').alias('win_count'))
home_count = futbol_team_info.groupBy(fn.col('home_team').alias('team'), 'game_year').agg(fn.count('home_team').alias('home_count'))
away_count = futbol_team_info.groupBy(fn.col('away_team').alias('team'), 'game_year').agg(fn.count('away_team').alias('away_count'))

# join them together
team_count = win_count.join(home_count, ['team', 'game_year']).join(away_count, ['team', 'game_year'])
team_count.show()

# generate total game counts and winning rate
team_count = team_count.withColumn('game_count', team_count['home_count'] + team_count['away_count'])
team_count = team_count.withColumn('win_rate', team_count['win_count'] / team_count['game_count'])

# Obtain the teams with the highest winning rate in each calander year
window = Window.partitionBy('game_year')
team_count.withColumn("max_win_rate", fn.max("win_rate").over(window)).filter("max_win_rate = win_rate").drop("max_win_rate").show()

+------------+--------------------+--------------------+--------------------+-------------------+--------------+--------------+-----------------+-----------------+-----------------+
|country_name|         league_name|           home_team|           away_team|               date|home_team_goal|away_team_goal|betting_odds_home|betting_odds_draw|betting_odds_away|
+------------+--------------------+--------------------+--------------------+-------------------+--------------+--------------+-----------------+-----------------+-----------------+
| Netherlands|Netherlands Eredi...|          SC Cambuur|                 PSV|2015-09-12 00:00:00|             0|             6|              6.5|              4.5|             1.44|
| Netherlands|Netherlands Eredi...|          PEC Zwolle|           Excelsior|2015-09-12 00:00:00|             3|             0|             1.73|              3.8|              4.5|
|      Poland|  Poland Ekstraklasa|       Śląsk Wrocław|Jagiellonia Biały...|2015-09-12 00

# Analysis of England & Germany Teams

In [25]:
sparkDF2016.write.partitionBy("country_name").mode("overwrite").parquet('matchBySeasonByTeam.parquet')

Now we have the 2015/2016 season partitioned by country. Next, we will load in the England and Germany parquet files so we can run our analysis on the matches from those countries.

In [26]:
sdf_England = spark.read.parquet("matchBySeasonByTeam.parquet/country_name=England")
sdf_Germany = spark.read.parquet("matchBySeasonByTeam.parquet/country_name=Germany")

In [27]:
import functools 
def unionAll(dfs):
    return functools.reduce(lambda df1,df2: df1.union(df2.select(df1.columns)), dfs)

unioned_df = unionAll([sdf_Germany, sdf_England])

count the number of matches in 2015/2016 season for Germany and England. Then check for duplicates and drop them.

In [28]:
#England
print("Count of England matches from 2015/2016 season: '%s'" % sdf_England.count())
print("Count of distinct England matches: '%s'" % sdf_England.distinct().count())

#Germany
print("\nCount of Germany matches from 2015/2016 season: '%s'" % sdf_Germany.count())
print("Count of distinct Germany matches: '%s'" % sdf_Germany.distinct().count())

unioned_df.count()

Count of England matches from 2015/2016 season: '380'
Count of distinct England matches: '380'

Count of Germany matches from 2015/2016 season: '306'
Count of distinct Germany matches: '306'


686

There are no duplicates, but we can still 'drop' them for good practice.

In [29]:
sdf_England = sdf_England.dropDuplicates()
sdf_Germany = sdf_Germany.dropDuplicates()

Next, we are going to look at the data

In [30]:
unioned_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- id: string (nullable = true)
 |-- league_name: string (nullable = true)
 |-- stage: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_team_goal: integer (nullable = true)
 |-- away_team_goal: integer (nullable = true)
 |-- goal: string (nullable = true)
 |-- shoton: string (nullable = true)
 |-- shotoff: string (nullable = true)
 |-- foulcommit: string (nullable = true)
 |-- card: string (nullable = true)
 |-- cross: string (nullable = true)
 |-- corner: string (nullable = true)
 |-- possession: string (nullable = true)
 |-- betting_odds_home: double (nullable = true)
 |-- betting_odds_draw: double (nullable = true)
 |-- betting_odds_away: double (nullable = true)



In [31]:
#Look at head
unioned_df.limit(5).toPandas().head(5)

,_c0,id,league_name,stage,date,home_team,away_team,home_team_goal,away_team_goal,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,betting_odds_home,betting_odds_draw,betting_odds_away
0,23100,10204,Germany 1. Bundesliga,4,2015-09-13 00:00:00,FC Schalke 04,1. FSV Mainz 05,2,1,<goal><value><comment>npm</comment><event_inci...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>62</comment><stats...,1.85,3.75,4.20
1,23101,10210,Germany 1. Bundesliga,4,2015-09-13 00:00:00,TSG 1899 Hoffenheim,SV Werder Bremen,1,3,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>57</comment><stats...,1.95,3.75,3.75
2,23134,10219,Germany 1. Bundesliga,5,2015-09-18 00:00:00,1. FSV Mainz 05,TSG 1899 Hoffenheim,3,1,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><event_incident_typefk>123</even...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>70</comment><stats...,2.15,3.50,3.30
3,23156,10213,Germany 1. Bundesliga,5,2015-09-19 00:00:00,VfL Wolfsburg,Hertha BSC Berlin,2,0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>54</comment><stats...,1.50,4.33,6.50
4,23157,10214,Germany 1. Bundesliga,5,2015-09-19 00:00:00,Hamburger SV,Eintracht Frankfurt,0,0,<goal />,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>55</comment><stats...,2.50,3.50,2.75


# England team analysis

Analyzing most winning teams for England teams

In [32]:
from pyspark.sql import functions as fn
from pyspark.sql import Window

#obtain match level information
team_info = ['league_name', 'home_team','away_team','date', 'home_team_goal', 'away_team_goal', 'betting_odds_home', 'betting_odds_draw', 'betting_odds_away']

futbol_team_info = sdf_England.select(team_info)

futbol_team_info = futbol_team_info.withColumn('win_team', fn.when(fn.col('home_team_goal') > fn.col('away_team_goal'), fn.col('home_team')).otherwise(fn.col('away_team')))
futbol_team_info = futbol_team_info.withColumn('year', fn.substring('date', 0, 4))
futbol_team_info.show()

win_count = futbol_team_info.groupBy(fn.col('win_team').alias('team'), 'year').agg(fn.count('win_team').alias('win_count'))
home_count = futbol_team_info.groupBy(fn.col('home_team').alias('team'), 'year').agg(fn.count('home_team').alias('home_count'))
away_count = futbol_team_info.groupBy(fn.col('away_team').alias('team'), 'year').agg(fn.count('away_team').alias('away_count'))

#join them together
team_count = win_count.join(home_count, ['team', 'year']).join(away_count, ['team', 'year'])
team_count.show()

# generate total game counts and winning rate
team_count = team_count.withColumn('game_count', team_count['home_count'] + team_count['away_count'])
team_count = team_count.withColumn('win_rate', team_count['win_count'] / team_count['game_count'])

#Obtain the teams with the highest winning rate in each calander year
window = Window.partitionBy('year')
team_count.withColumn("max_win_rate", fn.max("win_rate").over(window)).filter("max_win_rate = win_rate").drop("max_win_rate").show()

#Obtain the teams with the lowest winning rate in each calander year
window = Window.partitionBy('year')
team_count.withColumn("min_win_rate", fn.min("win_rate").over(window)).filter("min_win_rate = win_rate").drop("min_win_rate").show()

+--------------------+--------------------+--------------------+-------------------+--------------+--------------+-----------------+-----------------+-----------------+--------------------+----+
|         league_name|           home_team|           away_team|               date|home_team_goal|away_team_goal|betting_odds_home|betting_odds_draw|betting_odds_away|            win_team|year|
+--------------------+--------------------+--------------------+-------------------+--------------+--------------+-----------------+-----------------+-----------------+--------------------+----+
|England Premier L...|   Manchester United|          Sunderland|2015-09-26 00:00:00|             3|             0|              1.2|              7.5|             16.0|   Manchester United|2015|
|England Premier L...|West Bromwich Albion|      Crystal Palace|2016-02-27 00:00:00|             3|             2|              2.6|              3.1|              2.8|West Bromwich Albion|2016|
|England Premier L...|   

## Germany Team Analysis

Here, I am going to do the same thing as above but for Germany teams only

In [33]:
#obtain match level information
team_info = ['league_name', 'home_team','away_team','date', 'home_team_goal', 'away_team_goal', 'betting_odds_home', 'betting_odds_draw', 'betting_odds_away']

futbol_team_info = sdf_Germany.select(team_info)

futbol_team_info = futbol_team_info.withColumn('win_team', fn.when(fn.col('home_team_goal') > fn.col('away_team_goal'), fn.col('home_team')).otherwise(fn.col('away_team')))
futbol_team_info = futbol_team_info.withColumn('year', fn.substring('date', 0, 4))
futbol_team_info.show()

win_count = futbol_team_info.groupBy(fn.col('win_team').alias('team'), 'year').agg(fn.count('win_team').alias('win_count'))
home_count = futbol_team_info.groupBy(fn.col('home_team').alias('team'), 'year').agg(fn.count('home_team').alias('home_count'))
away_count = futbol_team_info.groupBy(fn.col('away_team').alias('team'), 'year').agg(fn.count('away_team').alias('away_count'))

#join them together
team_count = win_count.join(home_count, ['team', 'year']).join(away_count, ['team', 'year'])
team_count.show()

# generate total game counts and winning rate
team_count = team_count.withColumn('game_count', team_count['home_count'] + team_count['away_count'])
team_count = team_count.withColumn('win_rate', team_count['win_count'] / team_count['game_count'])

#Obtain the teams with the highest winning rate in each calander year
window = Window.partitionBy('year')
team_count.withColumn("max_win_rate", fn.max("win_rate").over(window)).filter("max_win_rate = win_rate").drop("max_win_rate").show()

#Obtain the teams with the lowest winning rate in each calander year
window = Window.partitionBy('year')
team_count.withColumn("min_win_rate", fn.min("win_rate").over(window)).filter("min_win_rate = win_rate").drop("min_win_rate").show()

+--------------------+-------------------+--------------------+-------------------+--------------+--------------+-----------------+-----------------+-----------------+-------------------+----+
|         league_name|          home_team|           away_team|               date|home_team_goal|away_team_goal|betting_odds_home|betting_odds_draw|betting_odds_away|           win_team|year|
+--------------------+-------------------+--------------------+-------------------+--------------+--------------+-----------------+-----------------+-----------------+-------------------+----+
|Germany 1. Bundes...|TSG 1899 Hoffenheim|    SV Werder Bremen|2015-09-13 00:00:00|             1|             3|             1.95|             3.75|             3.75|   SV Werder Bremen|2015|
|Germany 1. Bundes...|         1. FC Köln|Borussia Möncheng...|2015-09-19 00:00:00|             1|             0|              2.4|              3.3|              3.0|         1. FC Köln|2015|
|Germany 1. Bundes...|      VfL Wol

## Comparing Germany and England

In [35]:
#obtain match level information
team_info = ['league_name', 'home_team','away_team','date', 'home_team_goal', 'away_team_goal', 'betting_odds_home', 'betting_odds_draw', 'betting_odds_away']

futbol_team_info = unioned_df.select(team_info)

futbol_team_info = futbol_team_info.withColumn('win_team', fn.when(fn.col('home_team_goal') > fn.col('away_team_goal'), fn.col('home_team')).otherwise(fn.col('away_team')))
futbol_team_info = futbol_team_info.withColumn('year', fn.substring('date', 0, 4))
futbol_team_info.show()

win_count = futbol_team_info.groupBy(fn.col('win_team').alias('team'), 'year').agg(fn.count('win_team').alias('win_count'))
home_count = futbol_team_info.groupBy(fn.col('home_team').alias('team'), 'year').agg(fn.count('home_team').alias('home_count'))
away_count = futbol_team_info.groupBy(fn.col('away_team').alias('team'), 'year').agg(fn.count('away_team').alias('away_count'))

#join them together
team_count = win_count.join(home_count, ['team', 'year']).join(away_count, ['team', 'year'])
team_count.show()

# generate total game counts and winning rate
team_count = team_count.withColumn('game_count', team_count['home_count'] + team_count['away_count'])
team_count = team_count.withColumn('win_rate', team_count['win_count'] / team_count['game_count'])

#Obtain the teams with the highest winning rate in each calander year
window = Window.partitionBy('year')
team_count.withColumn("max_win_rate", fn.max("win_rate").over(window)).filter("max_win_rate = win_rate").drop("max_win_rate").show()

#Obtain the teams with the lowest winning rate in each calander year
window = Window.partitionBy('year')
team_count.withColumn("min_win_rate", fn.min("win_rate").over(window)).filter("min_win_rate = win_rate").drop("min_win_rate").show()

+--------------------+--------------------+--------------------+-------------------+--------------+--------------+-----------------+-----------------+-----------------+--------------------+----+
|         league_name|           home_team|           away_team|               date|home_team_goal|away_team_goal|betting_odds_home|betting_odds_draw|betting_odds_away|            win_team|year|
+--------------------+--------------------+--------------------+-------------------+--------------+--------------+-----------------+-----------------+-----------------+--------------------+----+
|Germany 1. Bundes...|       FC Schalke 04|     1. FSV Mainz 05|2015-09-13 00:00:00|             2|             1|             1.85|             3.75|              4.2|       FC Schalke 04|2015|
|Germany 1. Bundes...| TSG 1899 Hoffenheim|    SV Werder Bremen|2015-09-13 00:00:00|             1|             3|             1.95|             3.75|             3.75|    SV Werder Bremen|2015|
|Germany 1. Bundes...|   

We can see many things from this analysis. We fist determined the most winning team in England and then in Germany. We then analyzed teams from both germany and england together to determine which country has the best team. 